### 3. Analýza známek u studijní skupiny (gql_ug + gql_granting).

Jako parametr vezme definici kontingenční tabulky, GQL dotaz, dotaz zrealizuje, a data vloží do sešitu Excel spolu s kontingenční tabulkou. Vytvořte jako FastAPI službu.

Společné podmínky
- Vytvořit GQL dotaz na základě existující federace,
- Definovat transformaci GQL response -> table rows (vstup pro kontingenční tabulku)
- Vytvořit kontingenční tabulku
- Vytvořit koláčový / sloupcový graf
- Vytvořit Sunburst / Chord graf
- Výsledek realizujte jako ipynb notebook (autentizace jménem a heslem, realizace aiohttp, transformace response, vytvoření tabulky, vytvoření grafu).

In [1]:
import aiohttp
import asyncio
import json

In [2]:
gql_query = """
query {
	acClassificationPage {
  	id
  	student {
    	id
    	email
    	fullname
    	name
    	surname
  	}
  	semester {
    	id
    	order
    	subject {
      	id
      	name
      	program {
        	id
        	name
      	}
    	}
  	}
  	level {
    	id
    	name
  	}
		date
  	order
  }
}
"""

In [3]:
with open('data.json', 'r') as file:
    data = json.load(file)
    
print(data)

{'data': {'acClassificationPage': [{'id': 'ce250bd0-b095-11ed-9bd8-0242ac110002', 'student': {'id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'email': 'john.newbie@world.com', 'fullname': 'John Newbie', 'name': 'John', 'surname': 'Newbie'}, 'semester': {'id': 'ce250af4-b095-11ed-9bd8-0242ac110002', 'order': 1, 'subject': {'id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'name': 'Programovani', 'program': {'id': '2766fc9a-b095-11ed-9bd8-0242ac110002', 'name': 'IT Technologie'}}}, 'level': {'id': '5faea396-b095-11ed-9bd8-0242ac110002', 'name': 'F'}, 'date': '2023-04-19T08:00:00', 'order': 1}, {'id': 'ce250bd1-b095-11ed-9bd8-0242ac110002', 'student': {'id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'email': 'john.newbie@world.com', 'fullname': 'John Newbie', 'name': 'John', 'surname': 'Newbie'}, 'semester': {'id': 'ce250af4-b095-11ed-9bd8-0242ac110002', 'order': 1, 'subject': {'id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'name': 'Programovani', 'program': {'id': '2766fc9a-b095-11ed-9bd8-0242ac1

In [4]:
def convertLevel(level):
    row = {}
    
    row["level_id"] = level["level"]["id"]
    row["level_name"] = level["level"]["name"]
    row["level_order"] = level["order"]
    row["level_date"] = level["date"]
    
    row["student_id"] = level["student"]["id"]  
    row["student_email"] = level["student"]["email"]  
    
    row["subject_id"] = level["semester"]["subject"]["id"]  
    row["subject_name"] = level["semester"]["subject"]["name"]   
    row["semestr_order"] = level['semester']['order']
    
    row["program_id"] = level["semester"]["subject"]["program"]["id"]  
    row["program_name"] = level["semester"]["subject"]["program"]["name"]
    return row

converted_data = [convertLevel(level) for level in data["data"]["acClassificationPage"]]

sorted_data = sorted(converted_data, key=lambda x: x['level_order'], reverse=True)
filtered_data = sorted_data[:1]

print(filtered_data)

[{'level_id': '5faea21a-b095-11ed-9bd8-0242ac110002', 'level_name': 'C', 'level_order': 2, 'level_date': '2023-04-19T08:00:00', 'student_id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'student_email': 'john.newbie@world.com', 'subject_id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Programovani', 'semestr_order': 1, 'program_id': '2766fc9a-b095-11ed-9bd8-0242ac110002', 'program_name': 'IT Technologie'}]


In [5]:
with open('fake_data.json', 'r') as file:
    fake_data = json.load(file)
    
grouped_data = filtered_data + fake_data

print(grouped_data)

with open("grouped_data.json", "w") as json_file:
    json.dump(grouped_data, json_file)

[{'level_id': '5faea21a-b095-11ed-9bd8-0242ac110002', 'level_name': 'C', 'level_order': 2, 'level_date': '2023-04-19T08:00:00', 'student_id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'student_email': 'john.newbie@world.com', 'subject_id': 'ce250a68-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Programovani', 'semestr_order': 1, 'program_id': '2766fc9a-b095-11ed-9bd8-0242ac110002', 'program_name': 'IT Technologie'}, {'level_id': '6d5a1c9e-3db6-11ec-8d3d-0242ac130003', 'level_name': 'A', 'level_order': 3, 'level_date': '2023-05-20T09:30:00', 'student_id': '4f8e72aa-3db6-11ec-8d3d-0242ac130003', 'student_email': 'emma.smith@example.com', 'subject_id': 'ce250a69-b095-11ed-9bd8-0242ac110002', 'subject_name': 'Pocitacove site', 'semester_order': 1, 'program_id': '2766fc9a-b095-11ed-9bd8-0242ac110002', 'program_name': 'IT Technologie'}, {'level_id': '7e6b3f8a-3db6-11ec-8d3d-0242ac130003', 'level_name': 'B', 'level_order': 4, 'level_date': '2023-06-10T14:00:00', 'student_id': '5fbc249e-3db6-11